## 2x2 Vectors Graph

A simple model with two input layers and two output layers.

## Build and save the graph

Ensure you are working in a virtual environment with tensorflow >= 1.9 installed, and in the notebook's directory on the command line:

```bash
rm -r saved_graph
```

In [2]:
import tensorflow as tf

In [3]:
# construct the graph

x = tf.placeholder(tf.float32, shape=(1,4), name="input1")
y = tf.placeholder(tf.float32, shape=(1,4), name="input2")

w = tf.Variable([[1,0,1,0]], dtype=tf.float32, name="w")
b = tf.Variable([[0,1,0,1]], dtype=tf.float32, name="b")

xw = tf.matmul(x, tf.transpose(w))
yb = tf.matmul(y, tf.transpose(b))

z = tf.multiply( xw, yb, name="output1" )
s = tf.add( xw, yb, name="output2" )

# execute the graph

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    z_eval, s_eval = sess.run([z, s], feed_dict={x: [[1,2,3,4]], y: [[10,20,30,40]]})
    
    # saver.save(sess, "./saved_checkpoints/model.ckpt")
    # tf.train.write_graph(sess.graph, './saved_graph', 'graph.pb', as_text=False)
    tf.saved_model.simple_save(sess, "./saved_graph", inputs={"input1": x, "input2": y}, outputs={"output1": z, "output2": s})
    
    print(z_eval, s_eval)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_graph/saved_model.pb
[[240.]] [[64.]]


## Freeze The Graph

Having used the `tf.saved_model.simple_save` function, now use the `freeze_graph` cli. Tensorflow also provides a python interface to the freeze_graph function.

On the command line:

```bash
mkdir frozen_graph

freeze_graph \
    --input_saved_model_dir=saved_graph \
    --saved_model_tags=serve \
    --output_graph=frozen_graph/frozen_graph.pb \
    --output_node_names=output1,output2
```

## Convert the Model to a TensorFlow Lite Format

Use the `toco` command line utility to convert the frozen graph to a tensorflow lite compatible file:

```bash
mkdir tflite

toco \
	--graph_def_file=frozen_graph/frozen_graph.pb \
	--output_file=tflite/model.tflite \
	--input_arrays=input1,input2 \
	--input_shapes=1,4:1,4 \
	--output_arrays=output1,output2
```